In [23]:
import pandas as pd

In [24]:
data_path = "../conversations/dsat/voip-0f41c16f2f-20130325_192310.txt"

In [25]:
with open (data_path) as f:
    data = f.read()

In [26]:
print(data)

[SYSTEM] Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?
[USER]   i'm looking for a restaurant that serves german food
[SYSTEM] You are looking for a german restaurant right?
[USER]   no
[SYSTEM] Let me confirm , You are looking for a restaurant in the moderate price range right?
[USER]   yes
[SYSTEM] I'm sorry but there is no restaurant serving german food
[USER]   how about free
[SYSTEM] I'm sorry but there is no restaurant serving german food
[USER]   a restaurant serving french food
[SYSTEM] You are looking for a german restaurant right?
[USER]   thank you goodbye



In [251]:
def read_file(file):
    with open (file) as f:
        lines = [line.rstrip('\n') for line in f]
    id_name = file.split("/")[-1].split(".txt")[0]
    return id_name,lines

In [ ]:
def number_repetition(lines):
    new_list =[]
    for line in lines:
        if "SYSTEM" in line:
            new_list.append(line.strip())
    #create dictionary and count the repetition of sentence (Feature 1)
    new_dict = {}
    for li in new_list:
        if li in new_dict:
            new_dict[li] += 1
        else:
            new_dict[li] = 1
    num_rep = sum([value for value in new_dict.values() if value > 1])
    #Percentage of the repetition sentence in the entire conversation (Feature 3)
    num_rep_per = num_rep/len(new_list)
    
    return num_rep,num_rep_per

In [ ]:
def len_conversation(lines):
    return len(lines)

#### Repetition of the chatbot / 4 features

In [226]:
def number_repetition(file):
    with open (file) as f:
        lines = f.readlines()
        new_list =[]
        for line in lines:
            if "SYSTEM" in line:
                new_list.append(line.strip())
    #create dictionary and count the repetition of sentence (Feature 1)
    new_dict = {}
    for li in new_list:
        if li in new_dict:
            new_dict[li] += 1
        else:
            new_dict[li] = 1
    num_rep = sum([value for value in new_dict.values() if value > 1])
    
    #create a matrix to calculate repetition (Feature 2)
    m = [[0]* len(new_list) for i in range(len(new_list))]
    for i, x in enumerate(new_list):
        for j, y in enumerate(new_list):
            if x == y:
                m[i][j] = 1 
    all_value = 0
    for li in m:
        a = sum(li)
        all_value+=a
    num_rep_exp = (all_value-len(new_list))/2
    
    #Percentage of the repetition sentence in the entire conversation (Feature 3)
    num_rep_per = num_rep/len(new_list)
    #Number of sentences in the conversation (Feature 4)
    len_conversation = len(lines)
    
    return num_rep, num_rep_per,num_rep_exp,len_conversation

In [227]:
import glob
text_files_dsat = glob.glob("../conversations/dsat/*.txt")
text_files_sat = glob.glob("../conversations/sat/*.txt")

In [228]:
with open(text_files_dsat[6]) as f:
    print(f.read())

[SYSTEM] Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?
[USER]   yes can i have a restaurant who
[SYSTEM] What kind of food would you like?
[USER]   cheap
[SYSTEM] What kind of food would you like?
[USER]   european
[SYSTEM] Sorry there is no european restaurant in the cheap price range
[USER]   european food any price
[SYSTEM] Sorry there is no european restaurant in the cheap price range
[USER]   okay can i have european food
[SYSTEM] Sorry there is no european restaurant in the cheap price range
[USER]   any price range
[SYSTEM] Sorry there is no european restaurant in the cheap price range
[USER]   uh european food
[SYSTEM] Sorry there is no european restaurant in the cheap price range
[USER]   can i have european food
[SYSTEM] Sorry there is no european restaurant in the cheap price range
[USER]   can i have the in any price range
[SYSTEM] Sorry there is no european restaurant in the cheap pr

#### create feature dataframe for disatified dataset

In [229]:
columns = ['num_rep','num_rep_per','num_rep_exp','len_conversation', 'Is_satisfied']
df_rep_dsat = pd.DataFrame(index=range(len(text_files_dsat)), columns=columns)

In [230]:
for idx,file in enumerate(text_files_dsat):
    num_rep, num_rep_per,num_rep_exp,len_conversation = number_repetition(file)
    df_rep_dsat.iloc[idx] = pd.Series({'num_rep':num_rep, 'num_rep_per':num_rep_per,
                                  'num_rep_exp':num_rep_exp,'len_conversation':len_conversation, 'Is_satisfied':0})

#### create feature dataframe for satified dataset

In [231]:
columns = ['num_rep','num_rep_per','num_rep_exp','len_conversation', 'Is_satisfied']
df_rep_sat = pd.DataFrame(index=range(len(text_files_sat)), columns=columns)

In [232]:
for idx,file in enumerate(text_files_sat):
    num_rep, num_rep_per,num_rep_exp,len_conversation = number_repetition(file)
    df_rep_sat.iloc[idx] = pd.Series({'num_rep':num_rep, 'num_rep_per':num_rep_per,
                                  'num_rep_exp':num_rep_exp,'len_conversation':len_conversation, 'Is_satisfied':1})

In [233]:
df_rep_dsat.head()

,num_rep,num_rep_per,num_rep_exp,len_conversation,Is_satisfied
0,2,0.142857,1,28,0
1,11,0.55,19,40,0
2,5,0.625,4,16,0
3,0,0,0,6,0
4,8,0.571429,16,28,0


In [234]:
df_rep_sat.head()

,num_rep,num_rep_per,num_rep_exp,len_conversation,Is_satisfied
0,8,0.571429,28,28,1
1,0,0,0,16,1
2,0,0,0,10,1
3,18,0.72,32,50,1
4,0,0,0,12,1


In [235]:
print('Number of disatisfied files: {}'.format(len(df_rep_dsat)))
print('Number of satisfied files: {}'.format(len(df_rep_sat)))

Number of disatisfied files: 157
Number of satisfied files: 314


#### concat 2 dataframe

In [236]:
new_df = pd.concat([df_rep_dsat,df_rep_sat])

#### shuffle the dataframe

In [237]:
df_shuffle = new_df.sample(frac=1).reset_index(drop=True)

In [203]:
df_shuffle.to_csv("features.csv",index=False)

#### Logistic regression

In [271]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [278]:
X = df_shuffle.iloc[:,:-1]
y = df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [279]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [280]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [281]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[22 13]
 [14 69]]


In [282]:
y_pred_new = classifier.predict_proba(X_test)
y_pred_new2 = classifier.decision_function(X_test)

In [283]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.77


In [284]:
from pandas import *
import numpy as np
from scipy.stats.stats import pearsonr
import itertools

In [168]:
correlations = {}
columns = df_shuffle.columns.tolist()

for col_a, col_b in itertools.combinations(columns, 2):
    correlations[col_a + '__' + col_b] = pearsonr(df_shuffle.loc[:, col_a], df_shuffle.loc[:, col_b])

result = DataFrame.from_dict(correlations, orient='index')
result.columns = ['PCC', 'p-value']

print(result.sort_index())

                                     PCC        p-value
len_conversation__Is_satisfied -0.353808   2.460829e-15
num_rep__Is_satisfied          -0.477934   2.981528e-28
num_rep__len_conversation       0.858641  3.229487e-138
num_rep__num_rep_exp            0.758841   2.193505e-89
num_rep__num_rep_per            0.856732  5.911338e-137
num_rep_exp__Is_satisfied      -0.347836   7.670654e-15
num_rep_exp__len_conversation   0.603272   5.067019e-48
num_rep_per__Is_satisfied      -0.522149   2.668523e-34
num_rep_per__len_conversation   0.592383   5.972811e-46
num_rep_per__num_rep_exp        0.561749   1.586553e-40


#### Build the model with one variable

In [177]:
X = df_shuffle.iloc[:,:1]
y = df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [178]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [179]:
print('Accuracy of num_rep: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of num_rep: 0.70


In [175]:
X = df_shuffle.iloc[:,1:2]
y = df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = y_train.astype('int')
y_test = y_test.astype('int')
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)
print('Accuracy of num_rep_per: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of num_rep_per: 0.73


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [181]:
X = df_shuffle.iloc[:,2:3]
y = df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = y_train.astype('int')
y_test = y_test.astype('int')
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)
print('Accuracy of num_rep_exp: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of num_rep_exp: 0.72


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [184]:
X = df_shuffle.iloc[:,3:4]
y = df_shuffle.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = y_train.astype('int')
y_test = y_test.astype('int')
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)
print('Accuracy of len_conversation: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of len_conversation: 0.66


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
